<a href="https://colab.research.google.com/github/CaldeCrack/CC3001-Algoritmos-y-Estructuras-de-Datos/blob/main/Tareas/Tarea_5_MinMaxHeap_AndresCalderon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 2022-2 - Tarea 5: `Min_Max_Heap` -- Andrés Calderón Guardia


## Profesores: Nelson Baloian, Jérémy Barbay, Benjamín Bustos, Patricio Poblete

Según las condiciones de su sección, pueden entregar un link a su tarea en `colab` o subir un archivo en el formato Jupyter NoteBook (`.ipynb`). Puede crear todas las funciones auxiliares que requiera para implementar las funciones solicitadas. Para programar las funciones debe usar los *array* de *numpy*. Los únicos métodos de los objetos de tipo *array* que puede utilizar en su solución son los siguientes:

- El método *array* para crear un arreglo.
- Los métodos *zeros*, *ones* y *full* para inicializar un *array*.

No se puede utilizar ningún otro método disponible en módulos de Python.


# El Problema


Se trata de extender una implementación de un `Max_Heap` (tal como, por ejemplo, la de los [apuntes](https://colab.research.google.com/github/ivansipiran/AED-Apuntes/blob/main/05_Pilas_Colas_y_Colas_de_Prioridad.ipynb#scrollTo=Implementaci_n_usando_un_Heap), o la implementación [abajo](https://colab.research.google.com/drive/1cy50yuJZ6leVGdJg2iXNZV6SHtYTXisu#scrollTo=Implementaci_n_de_un_Max_Heap_que_dibujo_los_estados_intermedios) la cual permite dibujar el árbol correspondiendo al heap) en un `Min_Max_Heap` (tal como está definido, por ejemplo, en [Wikipedia](https://en.wikipedia.org/wiki/Min-max_heap)), que soporta tanto `extract_min` como `extract_max` en tiempo logarítmico en la cantidad de elementos en el `Min_Max_Heap`.

## Descripción formal de un `Min_Max_Heap`


Un `Min_Max_Heap` de $n$ elementos es muy similar a un `Min_Heap` o un `Max_Heap` en el sentido que es un árbol binario balanceado perfecto representado en las $n$ primeras posiciones de un arreglo. La diferencia con un `Min_Heap` o un `Max_Heap` es que para los nodos cuya distancia (largo del camino) a la raíz es par (e.g., $0,2,4,\ldots$) aplican las reglas de un `Min_Heap` (i.e., tiene valor menor o igual a los de todos sus descendientes), mientras que para los nodos cuya distancia a la raíz es impar (e.g., ($1,3,5,\ldots$) aplican las reglas de un `Max_Heap` (i.e., tiene valor mayor o igual a los de todos sus descendientes).



Para insertar un elemento en un `Min_Max_Heap`, se inserta al final del arreglo y se compara con el valor de su nodo "*padre*":

- si es inferior al valor de su nodo padre, entonces es inferior a todos los valores de nodos `max` (i.e., a distancia impar de la raíz), y se deben considerar únicamente los nodos ancestros de tipo `min` (i.e., a distancia par de la raíz) para *corregir* el `Min_Max_Heap`, usando el proceso de un `Min_Heap`; por el contrario,

- si es superior al valor de su nodo padre, entonces es superior a todos los valores de nodos `min` (i.e., a distancia par de la raíz), y se deben considerar únicamente los nodos ancestros de tipo `max` (i.e., a distancia impar de la raíz) para *corregir* el `Min_Max_Heap`, usando el proceso de un `Max_Heap`.



Para extraer el mínimo de un `Min_Max_Heap`, se saca la raíz, se reemplaza por el último elemento del arreglo, y se procede a *corregir* el `Min_Max_Heap` de manera simétrica al proceso de la inserción.  Para extraer el máximo de un `Min_Max_Heap`, se saca el hijo mayor de la raíz, se reemplaza dicho valor por el último elemento del arreglo, y se procede a *corregir* el `Min_Max_Heap` de manera simétrica al proceso de la inserción.


Para corregir un `Min_Max_Heap` tomaremos el caso en que el nodo actual está en un nivel min (par), el caso para un nodo en nivel max es análogo.  Luego de reemplazar la raíz se debe comparar el nodo que se insertó con sus hijo y los hijos de sus hijos y tomar el mínimo entre estos siete elementos, luego, tenemos tres casos:
1. El mínimo es el nodo actual, en ese caso el heap ya está balanceado y no hay que realizar más cambios.
1. El mínimo es un hijo del nodo actual, en este caso se debe hacer swap entre el nodo y el hijo, y ahí termina la corrección.
1. El mínimo es un nieto del nodo actual, en este caso se debe hacer swap entre el nodo y el nieto, y corregir recursivamente desde el nieto.


# Parte 1: `Min_Max_Heap_es_valido`


Antes de comenzar a programar, estudie bien la definición de un `Min_Max_Heap` y luego defina un metodo `Min_Max_Heap_es_valido` que, dado un arreglo de valores, retorna `True` si corresponde a un `Min_Max_Heap` y `False` sino.

In [ ]:
import numpy as np

def Min_Max_Heap_es_valido(arreglo):
    # Condiciones iniciales
    if len(arreglo) > 0: # Arreglo contiene al menos un elemento para poder chequear la primera condición
        if np.min(arreglo) != arreglo[0]: return False # Primera condición para un Min-Max Heap
    if len(arreglo) > 1: # Arreglo contiene al menos dos elementos para poder chequear por lo menos una de las dos comprobaciones para la segunda condición
        if len(arreglo) > 2: # Arreglo contiene al menos tres elementos para poder chequear la segunda condición
            if np.max(arreglo) != arreglo[1] and np.max(arreglo) != arreglo[2]: return False # Segunda condición para un Min-Max Heap
        else: # El arreglo tenía solo dos elementos
            if np.max(arreglo) != arreglo[1]: return False # Segunda condición para un Min-Max Heap
    # Comprobar el resto del Heap
    large = len(arreglo)
    for i in range(large): # Comprobar que cada nodo esté correcto
        index = large-1-i # Partir por el final del arreglo
        if index == 0: return True # Ya se comprobó el Heap completo
        if level(index+1)%2 == 0: # Caso nivel par
            if level(index+1) > 1: # Si se cumple que existe el abuelo compararlo
                if arreglo[parent(parent(index))] > arreglo[index]: return False # Nodo abuelo debe ser menor
            if arreglo[parent(index)] < arreglo[index]: return False # Nodo padre debe ser mayor
        else: # Caso nivel impar
            if level(index+1) > 1: # Si se cumple que existe el abuelo compararlo
                if arreglo[parent(parent(index))] < arreglo[index]: return False # Nodo abuelo debe ser menor
            if arreglo[parent(index)] > arreglo[index]: return False # Nodo padre debe ser mayor

# Función auxiliar para hallar el índice del nodo padre
def parent(i):
    if i == 0: return 0
    return (i-1)//2

# Función auxiliar para hallar el nivel de un nodo
def level(i): return int(np.log2(i))

a = np.array([1,2,3])
assert Min_Max_Heap_es_valido(a) == True

a = np.array([1,2,3,4,5,6,7])
assert Min_Max_Heap_es_valido(a) == False

a = np.array([1,6,7,4,5,2,3])
assert Min_Max_Heap_es_valido(a) == True

a = np.array([8,71,41,31,10,11,16,46,51,31,21,13])
assert Min_Max_Heap_es_valido(a) == True

# Parte 2: `Min_Max_Heap`



A continuación implemente su solución como una clase `Min_Max_Heap` con las operaciones `insert`, `extract_max`, `extract_min`. Ejecute las pruebas señaladas más abajo y agregue otras pruebas inventadas por usted.

In [ ]:
import numpy as np

# ---------------------- Funciones auxiliares ----------------------
def trepar_max(a,i):# El elemento a[i] trepa hasta su nivel de prioridad como un Max Heap en un Min-Max Heap
    while i >= 3 and a[i] > a[parent(parent(i))]: # comparar con nodo abuelo
        (a[i],a[parent(parent(i))]) = (a[parent(parent(i))],a[i]) # intercambiamos con el abuelo
        i = parent(parent(i)) # subimos al lugar del abuelo

def trepar_min(a,i): # El elemento a[i] trepa hasta su nivel de prioridad como un Min Heap en un Min-Max Heap
    while i >= 3 and a[i] < a[parent(parent(i))]: # comparar con nodo abuelo
        (a[i],a[parent(parent(i))]) = (a[parent(parent(i))],a[i]) # intercambiamos con el abuelo
        i = parent(parent(i)) # subimos al lugar del abuelo

def hundir_min(a,i,n): # El elemento a[i] se hunde hasta su nivel de prioridad
    while 2*i+1 < n: # Mientras tenga al menos 1 hijo
        j = 2*i+1 # El hijo izquierdo
        k = j+1 # El hijo derecho
        # Nietos
        j1 = j*2+1
        j2 = j1+1
        k1 = k*2+1
        k2 = k1+1
        # Si algunos hijos/nietos no existen reemplazar su valor con el del primer hijo (único existente bajo la condición del while)
        if k >= n: k = j
        if j1 >= n: j1 = j
        if j2 >= n: j2 = j
        if k1 >= n: k1 = j
        if k2 >= n: k2 = j
        change = np.min([a[i], a[j], a[k], a[j1], a[j2], a[k1], a[k2]]) # Sacar el mínimo de entre los siete posibles nodos
        if change == a[i]: break # Caso el nodo cambiado ya es el mínimo
        (a[i],a[list(a[1:]).index(change)+1]) = (a[list(a).index(change)],a[i]) # Intercambiar valores
        if j1 < n: # Caso recursivo
            if change == a[j1]: i = j1
            elif change == a[j2]: i = j2
            elif change == a[k1]: i = k1
            elif change == a[k2]: i = k2
        break

def hundir_max(a,i,n): # El elemento a[i] se hunde hasta su nivel de prioridad
    while 2*i+1 < n: # Mientras tenga al menos 1 hijo
        j = 2*i+1 # El hijo izquierdo
        k = j+1 # El hijo derecho
        # Nietos
        j1 = j*2+1
        j2 = j1+1
        k1 = k*2+1
        k2 = k1+1
        # Si algunos hijos/nietos no existen reemplazar su valor con el del primer hijo (único existente bajo la condición del while)
        if k >= n: k = j
        if j1 >= n: j1 = j
        if j2 >= n: j2 = j
        if k1 >= n: k1 = j
        if k2 >= n: k2 = j
        change = np.max([a[i], a[j], a[k], a[j1], a[j2], a[k1], a[k2]]) # Sacar el máximo de entre los siete posibles nodos
        if change == a[i]: break # Caso el nodo cambiado ya es el máximo
        (a[i],a[list(a[i+1:]).index(change)+i+1]) = (a[list(a).index(change)],a[i]) # Intercambiar valores
        if j1 < n: # Caso recursivo
            if change == a[j1]: i = j1
            elif change == a[j2]: i = j2
            elif change == a[k1]: i = k1
            elif change == a[k2]: i = k2
        break

class Min_Max_Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn)
        self.n=0
    def insert(self,x):
        self.a[self.n] = x # Insertar valor al final del array
        self.n += 1 # Aumentar en uno la cantidad de elementos
        index = self.n-1 # Índice del array
        if self.a[index] > self.a[parent(index)]: # Caso valor mayor a su padre
            if level(self.n)%2 == 0: # Si el nodo está en un nivel par entonces intercambiar con el padre
                (self.a[index],self.a[parent(index)]) = (self.a[parent(index)],self.a[index]) # intercambio
                index = parent(index) # cambiar índice al del padre
            if index >= 3: trepar_max(self.a,index) # si el índice indica que existe al menos su nodo abuelo usar la función trepar_max
        else: # Caso valor menor a su padre
            if level(self.n)%2 == 1: # Si el nodo está en un nivel impar entonces intercambiar con el padre
                (self.a[index],self.a[parent(index)]) = (self.a[parent(index)],self.a[index]) # intercambio
                index = parent(index) # cambiar índice al del padre
            if index >= 3: trepar_min(self.a,index) # si el índice indica que existe al menos su nodo abuelo usar la función trepar_max
    def extract_max(self):
        assert self.n > 0 # Arreglo debe tener al menos un elemento para realizar esta operación
        if self.n == 1: # Arreglo contiene un solo elemento
            x = self.a[0]
            return x
        else: # Arreglo contiene al menos dos elementos
            if self.n > 2: # Arreglo contiene al menos tres elementos
                # Cálculo del mínimo
                if self.a[1] < self.a[2]:
                    x = self.a[2]
                    self.a[2] = self.a[self.n-1] # Movemos el elemento sobrante hacia el casillero libre
                    hundir_max(self.a[0:self.n-1],2,self.n-1) # Corregimos el Heap
                else:
                    x = self.a[1]
                    self.a[1] = self.a[self.n-1] # Movemos el elemento sobrante hacia el casillero libre
                    hundir_max(self.a[0:self.n-1],1,self.n-1) # Corregimos el Heap
                self.n -= 1 # Achicamos el Heap
                return x
            else: # El arreglo tenía solo dos elementos
                x = self.a[1]
                return x
    def extract_min(self):
        assert self.n > 0 # Arreglo debe tener al menos un elemento para realizar esta operación
        x = self.a[0] # Esta variable lleva el máximo, el casillero 0 queda vacante
        self.n -= 1 # Achicamos el Heap
        self.a[0] = self.a[self.n] # Movemos el elemento sobrante hacia el casillero libre
        hundir_min(self.a[0:self.n],0,self.n) # Corregimos el Heap
        return x
    def imprimir(self):
        print(self.a[0:self.n])

h = Min_Max_Heap(14)
h.insert(1)
h.insert(10)
h.insert(2)
h.insert(9)
h.insert(3)
h.insert(8)
h.insert(4)
h.insert(7)
h.insert(5)
h.insert(6)
h.insert(90)
h.insert(52)
h.insert(15)
h.insert(13)
h.imprimir()
assert Min_Max_Heap_es_valido(h.a), h.a
min = h.extract_min()
assert min == 1, min
h.imprimir()
max = h.extract_max()
assert max == 90, max
h.imprimir()

[ 1. 90. 52.  5.  3.  2.  4.  9.  7.  6. 10.  8. 15. 13.]
[ 2. 90. 52.  5.  3. 13.  4.  9.  7.  6. 10.  8. 15.]
[ 2. 15. 52.  5.  3. 13.  4.  9.  7.  6. 10.  8.]


# Para los curiosos

## Implementación de un `Max_Heap` que dibujo los estados intermedios.

En caso de que usted desee agregarle visualización a su  solución, a continuación se muestra cómo visualizar la operación de un heap común y corriente. Usted puede usar este código como base o como inspiración para una implementación visualizada de lo que se pide en esta tarea.

In [ ]:
import graphviz
from IPython.display import display_svg, SVG,display

import numpy as np

def dibuja_heap(heap,n,dot=graphviz.Digraph(engine="dot",format="svg"),prefix=""):
    "Dibuja un heap dado como un arreglo."
    position_of_last_internal_node = n//2
    for i,v in enumerate(heap[:n]):
        dot.node(prefix+str(i),str(v))
    for i in range(position_of_last_internal_node-1):
        dot.edge(prefix+str(i),prefix+str(2*(i+1)-1))
        dot.edge(prefix+str(i),prefix+str(2*(i+1)))
    if n>1:
        dot.edge(prefix+str(position_of_last_internal_node-1),prefix+str(2*(position_of_last_internal_node)-1))
        if 2*position_of_last_internal_node<n:
        dot.edge(prefix+str(position_of_last_internal_node-1),prefix+str(2*position_of_last_internal_node))
    return dot

class Max_Heap:
    def __init__(self,maxn=100,que_dibujar="todo"):
        self.a=np.zeros(maxn)
        self.n=0
        self.dot = graphviz.Digraph(engine="dot",format="svg")
        self.time=0
        self.que_dibujar = que_dibujar
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x
        a.trepar(self.n)
        self.n+=1
        if self.que_dibujar != "nada":
          self.capture_the_moment()

    def extract_max(self):
        assert self.n>0
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        a.hundir(0,self.n)
        if self.que_dibujar != "nada":
          self.capture_the_moment()
        return x

    def imprimir(self):
        print(self.a[0:self.n])
    def trepar(self,j): # El elemento a[j] trepa hasta su nivel de prioridad
        while j>=1 and self.a[j]>self.a[(j-1)//2]:
            (self.a[j],self.a[(j-1)//2])=(self.a[(j-1)//2],self.a[j]) # intercambiamos con el padre
            j=(j-1)//2 # subimos al lugar del padre
            if self.que_dibujar == "todo":
              self.capture_the_moment()
    def hundir(self,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
        while 2*j+1<n: # mientras tenga al menos 1 hijo
            k=2*j+1 # el hijo izquierdo
            if k+1<n and self.a[k+1]>self.a[k]: # el hijo derecho existe y es mayor
                k+=1
            if self.a[j]>=self.a[k]: # tiene mejor prioridad que ambos hijos
                break
            (self.a[j],self.a[k])=(self.a[k],self.a[j]) # se intercambia con el mayor de los hijos
            j=k # bajamos al lugar del mayor de los hijos
            if self.que_dibujar == "todo":
              self.capture_the_moment()

    def capture_the_moment(self):
      moment = "---"+str(self.time)+"---"
      self.dot.node(repr(moment),"t="+str(self.time))
      self.dot.edge(repr(moment),moment+"0") #Edge to the root of the heap
      self.dot = dibuja_heap(heap=self.a,n=self.n,dot=self.dot,prefix=moment)
      self.time+=1

    def dibuja(self):
      svg = self.dot.pipe().decode('utf-8')
      return SVG(svg)


IndentationError: ignored

Uno puede dibujar el estado del heap despues de cada insercion y extraccion:

In [ ]:
a=Max_Heap(10,que_dibujar="inserciones y extracciones")
a.insert(1)
a.insert(2)
a.insert(3)
a.insert(4)
a.insert(5)
a.insert(6)
a.insert(7)
a.insert(8)
a.insert(9)
a.insert(10)
a.dibuja()

O uno puede dibujar el estado del heap después de cada movimiento de elemento (e.g. dentro de `hundir`):
En el ejemplo abajo, observen cómo el elemento "10" está insertado "abajo" del heap, y progresivamente sube en el árbol.

In [ ]:
a=Max_Heap(10,que_dibujar="nada")
a.insert(1)
a.insert(2)
a.insert(3)
a.insert(4)
a.insert(5)
a.insert(6)
a.insert(7)
a.insert(8)
a.insert(9)
a.que_dibujar = "todo"
a.insert(10)
a.dibuja()